In [24]:
import pandas as pd
import time
import os
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By # 셀레니움 4.0부터 포함된 객체(모듈)
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [51]:
def setting(stock_code):
    # 드라이버 설정
    global driver
    chrome_options = webdriver.ChromeOptions() # 크롬브라우저를 컨트롤하는 드라이버 옵션 생성
    chrome_options.add_experimental_option("detach", True)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    # url 로딩
    url = f"https://m.stock.naver.com/domestic/stock/{stock_code}/discuss"
    driver.get(url)
    time.sleep(3)

In [71]:
stocks = {
    '096770': 'SK이노베이션',
    #'005930': '삼성전자',
    #'035420': 'NAVER'
    }

for stock_code in stocks.keys():

    global driver
    chrome_options = webdriver.ChromeOptions() # 크롬브라우저를 컨트롤하는 드라이버 옵션 생성
    chrome_options.add_experimental_option("detach", True)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    # url 로딩
    url = f"https://m.stock.naver.com/domestic/stock/{stock_code}/discuss"
    driver.get(url)
    time.sleep(1)


In [76]:
body = driver.find_element(By.TAG_NAME, "body")
body.send_keys(Keys.END)
time.sleep(0.5)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#<button type="button" class="InfinityMoreButton_button__ETrlQ" data-nlogs="nbo.moreb"><span class="blind">게시글</span>더보기<span class="InfinityMoreButton_icon__641_T"></span></button>

In [77]:
try:
    old_df = pd.read_csv(f"naver_{stocks[stock_code]}.csv")
    existing_ids = set(old_df["post_id"].apply(str))
except FileNotFoundError:
    pass

collected_comments = []

body = driver.find_element(By.TAG_NAME, "body")
body.send_keys(Keys.END)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
    
comment_elements = driver.find_elements(By.CSS_SELECTOR, 'li.DiscussListItem_item__wzLRv')

for comment in comment_elements:
    post_id = str(comment.get_attribute("id"))

    if post_id in existing_ids: 
        break
    else:
        pass
    title = comment.find_element(By.CSS_SELECTOR, 'strong.DiscussListItem_title__LVPk5').text
    content = comment.find_element(By.CSS_SELECTOR, 'p.DiscussListItem_text__OKI6X').text
    
    collected_comments.append((post_id, title, content))
        

In [ ]:
if not collected_comments:
    print(f"{stocks[stock_code]}: 저장할 새 댓글 없음")
else: 
    df_new = pd.DataFrame(collected_comments, columns=["post_id", "title", "content"])
    df_new["platform"] = "네이버증권"
    df_new["stock_name"] = stocks[stock_code]
    df_new = df_new[["platform", "stock_name", "post_id", "title", "content"]]
    file_path = f"naver_{stocks[stock_code]}.csv"

140

In [61]:
driver.quit()

In [33]:
# def save_comments(stock_code):
if not collected_comments:
    print(f"{stocks[stock_code]}: 저장할 새 댓글 없음")
else: 
    df_new = pd.DataFrame(collected_comments, columns=["post_id", "title", "content"])
    df_new["platform"] = "네이버증권"
    df_new["stock_name"] = stocks[stock_code]
    df_new = df_new[["platform", "stock_name", "post_id", "title", "content"]]
    file_path = f"naver_{stocks[stock_code]}.csv"

if os.path.exists(file_path):
    df_existing = pd.read_csv(file_path, encoding="utf-8-sig")
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    df_combined = df_combined.drop_duplicates(subset=["post_id"], keep="first")
    df_combined.to_csv(file_path, index=False, encoding="utf-8-sig")
else:
    df_new.to_csv(file_path, index=False, encoding="utf-8-sig")


In [34]:
df = pd.read_csv('naver_SK이노베이션.csv')
df.info()
df.head()
df.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   platform    400 non-null    object
 1   stock_name  400 non-null    object
 2   post_id     400 non-null    int64 
 3   title       400 non-null    object
 4   content     400 non-null    object
dtypes: int64(1), object(4)
memory usage: 15.8+ KB


,platform,stock_name,post_id,title,content
0,네이버증권,SK이노베이션,298991300,적당히 하자,장난질이\n과하다\n눈에 보인다
1,네이버증권,SK이노베이션,298991300,한번더 속아보자! 주총일이나 1분기 실적발표 보고 판단하는게 좋을듯?,"나도 욕나오지만, 참고 기다려 볼려고\n합니다! 주총때 또 얼마나 좋은말 할지!\n..."
2,네이버증권,SK이노베이션,298991300,단타,먹이 ㅡ
3,네이버증권,SK이노베이션,298991300,형아는 SDI 평단 44~46 하던 거 나왔어,탈출?\n.\n걍 나온거지...
4,네이버증권,SK이노베이션,298991300,13만,다시 올려놔라


,platform,stock_name,post_id,title,content
395,네이버증권,SK이노베이션,298972200,어제 시초가 덥석 물었다가,여러날 고생하게 생겼네
396,네이버증권,SK이노베이션,298972200,좋은 하루 보내라 !,아침에 땀 빼고 오니 좋다
397,네이버증권,SK이노베이션,298972200,시브레 열받아서 오늘 다시 투입각 살핀다...,마5%만 떨궈줘 억따리 투입해서 작살 내줄께...\n어제 엇박자 걸려서 손절 짜증 ...
398,네이버증권,SK이노베이션,298972200,■대한민국의 반국가세력 공부하자,"■대한민국의 반국가세력 공부하자\n민주당 종북좌파,민주노총,전교조\n위 단체가 대한..."
399,네이버증권,SK이노베이션,298972200,오늘도,뺑이 틀겠네
